<a href="https://colab.research.google.com/github/roquearmas/pruebasIA/blob/main/Copia_de_STRIPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Planificación de STRIPS

Jesús Roque Armas Martín

Se plantea un escenario en el que hay cuatro bloques colocados unos sobre otros en estructura BACD siendo B el de más arriba y D el de abajo. Los bloques tienen tres posiciones horizontales donde colocarse. Se pretende que un brazo robótico, mediante planificación STRIPS, sea capaz de colocarlos, en orden ABCD, siendo A el bloque que quede en la posición superior y D en la inferior.

Voy a definir un vector de situación que refleje dónde está el robot y como están las tres posiciones. {R,B1,B2,B3} siendo Bx la representación de los bloques que haya en cada posición. De esta manera, la representación del estado inicial sería {2,BACD,0000,0000} y la del final {3,0000,0000,ABCD}

Para operar en un lenguaje numérico, voy a sustituir los valores de las letras por números, así :
A=1
B=2
C=3
D=4

Para representar el estado inicial el vector sería {2213400000000}
y el estado final , {3000000001234}


In [92]:
import numpy as np
max_profundidad=25 # Operaciones máximas a recordar en la pila
max_operaciones=20 # número de operaciones distintas
estado=np.zeros((max_operaciones,27))     # Pila de estados
operaciones=np.zeros(max_profundidad,dtype=int)  # Pila de operaciones
texto=['MoverBrazo(2,1)','MoverBloque(B,1,2)','MoverBrazo(2,1)','MoverBloque(A,1,2)','MoverBrazo(2,1)','MoverBloque(C,1,2)',
       'MoverBrazo(2,1)','MoverBloque(D,1,2)','MoverBloque(D,2,3)','MoverBrazo(3,2)','MoverBloque(C,2,3)','MoverBrazo(3,2)',
       'MoverBloque(A,2,1)','MoverBrazo(1,2)','MoverBloque(B,2,3)','MoverBrazo(3,2)','MoverBrazo(2,1)','moverBloque(A,1,2)','moverBloque(A,2,3)','moverBrazo(3,2)']
#PC=np.arange(max_operaciones*27) # Lista de propiedades PC para poder realizar la acción
PC=np.zeros((max_operaciones,27),dtype=int)
E=np.zeros((max_operaciones,27),dtype=int)
A=np.zeros((max_operaciones,27),dtype=int)
#E=np.arange(max_operaciones*27)  # Lista de propiedades a Eliminar de cada acción
#A=np.arange(max_operaciones*27)  # Lista de propiedades a Añadir de cada acción
inicio=np.arange(27)
fin=np.arange(27)
#         Robot|  A  |B C D|  B  |A C D|  C  |A B D|  D  |A B C
#         1 2 3|1 2 3|     |1 2 3|     |1 2 3|     |1 2 3|
inicio= [0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1] # (R,A,B,C,D) Robot está en 2, en la posición 1, todas las cajas , de arriba  a abajo BACD
#después de las posiciones relativas al brazo del robot, hay 6 dígitos por bloque. Para el cada uno de ellos, los,
fin   = [0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1]  # (R,B1,,B2,B3) Robot está en 3, en la posición 1, y 2 no hay nada y en la posición 3, ABCD



Defino las operaciones posibles, según las operaciones a realizar




In [93]:
PC[0]=np.array(inicio).astype(int)

#MoverBrazo[2,1]
E[0]=[1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
A[0]=[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

PC[1]=[1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1]
#MoverBloque (B,1,2)
E[1]= [0,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
A[1]= [0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
#          BR |  A  |  B  |  C  |  D
#leyenda   1,231,23BCD1,23ACD1,23ABD1,23ABC
PC[2]=[0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1]

#MoverBrazo (2,1)
E[2]= [1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
A[2]= [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

PC[3]=[1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1]
#MoverBloque (A,1,2)
E[3]= [0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1]
A[3]= [0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

PC[4]=[0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1]
#MoverBrazo (2,1)
E[4]= [1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
A[4]= [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

PC[5]=[1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1]
#moverBloque(C,1,2)
E[5]= [1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0]
A[5]= [0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]

PC[6]=[0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0]
#moverBRAZO(2,1,)
E[6]= [1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
A[6]= [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

PC[7]=[1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0]

E[7] =[0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1]
A[7] =[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0]

PC[8]=[0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0]
#moverBloque(D,2,3)
E[8]= [1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1]
A[8]= [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]



PC[9]=[0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0]
#moverBrazo(3,2)
E[9]= [1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
A[9]= [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

PC[10]=[0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0]
#moverBloque(C,2,3)
#           BR |  A  |  B  |  C  |  D
#leyenda    1,231,23BCD1,23ACD1,23ABD1,23ABC
E[10]= [1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1]
A[10]= [0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1]
#           BR |  A  |  B  |  C  |  D
#leyenda    1,231,23BCD1,23ACD1,23ABD1,23ABC
PC[11]=[0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1]
#moverBrazo(3,2)
E[11]= [1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
A[11]= [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]


PC[12]=[0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1]

#moverBloque(A,2,1,)
E[12]= [1,0,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
A[12]= [1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

PC[13]=[1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1]
#MoverBrazo (1,,2)
E[13]= [0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
A[13]= [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

PC[14]=[0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1]

#MoverBloque(B,2,3)
E[14]= [1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
A[14]= [0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]

PC[15]=[0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1]

#moverBrazo(3,2)
E[15]= [1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
A[15]= [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

PC[16]=[0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1]

#MoverBrazo (2,1)
E[16]= [1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
A[16]= [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

PC[17]=[1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1]
#moverBloque(A,1,2)
#           BR |  A   |  B   |  C   |  D
#leyenda    123 123BCD 123ACD 123ABD 123ABC
E[17]= [0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
A[17]= [0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

PC[18]=[0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1]

#moverBloque(A,2,3)
E[18]= [1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
A[18]= [0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

PC[19]=[0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1]

#moverBrazo(3,2) - Añado esta operación para que no casque el bucle while
E[19]= [1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
A[19]= [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]





Metodo para intentar una acción

In [96]:
import ipdb

PC=np.array( [0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1]).astype(int)
#moverBrazo(3,2)
E =np.array( [1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]).astype(int)
A =np.array( [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]).astype(int)

resta=PC & E # Eliminamos propiedades E
print('AND:' ,resta)
suma =resta | A  # Añade propiedades A
print('OR :',suma)

#PC[12]=[0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1]

ModuleNotFoundError: ignored

Metodo para comprobar que no hemos pasado por ese estado


In [86]:
def encontrado(siguiente):
  encontrado=False
  t=top
  while (t>=0):
      if np.array_equal(estado[t],siguiente):
      #if (estado[t]==siguiente):
          encontrado=True
      t-=1
  return encontrado

Metodo para comparar números binarios

In [79]:
def ratataacompara(vector1, vector2):

  # Convertimos los números binarios a objetos de bytes.

  numero_1_bytes = bin(vector1).encode('utf-8')
  numero_2_bytes = bin(vector2).encode('utf-8')

  # Comparamos los números bit a bit.

  resultado = 0
  for i in range(len(numero_1_bytes)):
    if numero_1_bytes[i] == numero_2_bytes[i]:
      resultado = resultado
    elif numero_1_bytes[i] == b'1':
      resultado = 1
    else:
      resultado = -1
  return resultado


In [94]:
def intentar(nodo,op):
   #pasar nodo a array
    npNodo=np.array(nodo).astype(int)

    if np.array_equal(npNodo & np.array(PC[op]),np.array(PC[op])): # Cumple las PC para aplicar la op
        print('Operación : ', op)
        print('---------------------------------------')
        print( "cumple con:",PC[op], texto[op])
        sigu=npNodo & E[op] # Eliminamos propiedades E
        print("Eliminamos ",E[op])
        print(" y queda:  ",sigu)
        sigu=sigu | A[op]  # Añade propiedades A
        print("Añadimos   ",A[op])
        print(" y queda:  ",sigu)
        print('---------------------------------------')
    else:
        sigu=np.zeros(27)
    return sigu

Programa principal

In [95]:
top=0 # Cima de la pila

estado[top]=inicio
operaciones[top]=0 # Las operaciones van desde 0 hasta max_operaciones-1
op=0
actual=np.array(inicio).astype(int)

while (top>=0): # Mientras queden estados por explorar
    while ((op<max_operaciones) and (top<max_profundidad-1) and not np.array_equal(np.array(actual).astype(int),np.array(fin))):
        siguiente=intentar(np.array(actual).astype(int),op) # Intenta realizar la acción op sobre el estado actual
        if not np.array_equal(siguiente,np.zeros(27)): # acción no permitida
            if (not(encontrado(siguiente))):# Si no hemos pasado por ese estado
                top+=1 # Almacenamos en la pila el estado y la acción
                estado[top]=np.array(actual).astype(int)
                operaciones[top]=op
                #print('/',top,texto[operaciones[top]],'-->',bin(estado[top]),end='')
                # Saltamos al siguiente estado
                actual=np.array(siguiente).astype(int)
                op=-1 # Para que empiece en 0
        op+=1
    if np.array_equal(np.array(actual).astype(int) & np.array(fin),np.array(fin)):
        print("SOLUCIÓN:")
        t=1
        while (t<top):
            print(texto[operaciones[t]],"-->",end='')
            t+=1
        print(texto[operaciones[t]],end='')
        top=0
    actual=estado[top]
    op=operaciones[top]+1
    top-=1

Operación :  0
---------------------------------------
cumple con: [0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1] MoverBrazo(2,1)
Eliminamos  [1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 y queda:   [0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1]
Añadimos    [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 y queda:   [1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1]
---------------------------------------
Operación :  1
---------------------------------------
cumple con: [1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1] MoverBloque(B,1,2)
Eliminamos  [0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 y queda:   [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1]
Añadimos    [0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 y queda:   [0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1]
---------------------------------------
Operación :  2
---------------------------------------
cumple con: [0 1 0 1 0 0 0 0 0 0 1 0 0 0